In [7]:
from ultralytics import YOLO
import cv2
from collections import OrderedDict
import numpy as np
import yaml
import os

os.environ['CLEARML_CONFIG_FILE'] = "clearml.conf"

from clearml import Task, InputModel

with open('config.yaml','r') as f:
    configModel = yaml.safe_load(f)

In [8]:
task = Task.init(project_name=configModel["clearml-project-config"]["project-name"], task_name=configModel["clearml-project-config"]["task-name"],
                 task_type=configModel["clearml-project-config"]["task-type"], reuse_last_task_id=configModel["clearml-project-config"]["id"])

inputModel = InputModel(project=configModel["clearml-project-config"]["project-name"], name=configModel["model-config"]["YOLO-model"],
                        only_published=configModel["model-config"]["published"], tags=configModel["model-config"]["tags"])

task.connect(inputModel)
pathToModel = inputModel.get_local_copy()

model = YOLO(pathToModel)
gcv_api_key_path = configModel["model-config"]["vision-key"]
imgSize = configModel["model-config"]["image-size"]

2023-05-12 21:15:25,536 - clearml.model - INFO - Selected model id: 9d5e897f0c44442e852d44b450e357fb


In [9]:
img = cv2.imread("vibrio/vibrio_mae/3-52-vibrio.jpg")

print(img.shape)

scalingH, scalingW = img.shape[0]/imgSize, img.shape[1]/imgSize
data = cv2.resize(img, (imgSize, imgSize))

results = model.predict(data, imgsz = imgSize,
                        conf = configModel["model-config"]["conf"], iou = configModel["model-config"]["iou"],
                        save = configModel["model-config"]["save-mode"], save_conf = configModel["model-config"]["save-mode"],
                        save_crop = configModel["model-config"]["save-mode"], save_txt = configModel["model-config"]["save-mode"],
                        device = configModel["model-config"]["device-mode"])

(3000, 4000, 3)



0: 640x640 21 greens, 31 yellows, 881.6ms
Speed: 3.0ms preprocess, 881.6ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [10]:
classes = results[0].names
print(classes)

{0: 'black', 1: 'green', 2: 'yellow'}


In [12]:
totalFound = {}
for boxes in results[0].boxes:
    for box in boxes:
        if int(box.cls) in totalFound.keys():
            totalFound[int(box.cls)] = totalFound[int(box.cls)] + 1
        else:
            totalFound[int(box.cls)] = 1

In [13]:
print(totalFound)

{1: 21, 2: 31}


In [14]:
finalDict = {}
dictDataEntity = {}
orderedDict = OrderedDict(sorted(totalFound.items()))
for key, value in orderedDict.items():
    for classKey, classValue in classes.items(): 
        if key == classKey:
             finalDict[classValue] = value

print(finalDict)

{'green': 21, 'yellow': 31}


In [23]:
for label in classes.values():
        if label not in finalDict.keys():
            finalDict[label] = "0"

In [15]:
finalDict

{'green': 21, 'yellow': 31}

In [18]:
from IPython.display import Image

In [19]:
display(Image(data=results[0].plot()))

2023-05-12 22:25:09,927 - clearml.Task - WARNING - ### TASK STOPPED - USER ABORTED - STATUS CHANGED ###


: 